# Imports

In [ ]:
!pip install yfinance

In [ ]:
# IMPORTS
import requests
import numpy as np
import pandas as pd
import re

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date



# Question 1: IPO Filings Web Scraping and Data Processing
What's the total sum ($m) of 2023 filings that happened on Fridays?

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/ Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs). Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given). You may be inspired by the function extract_numbers() in [Code Snippet 4], or you can write your own function to "parse" a string. Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all filings during 2023, which happened on Fridays (Date.dt.dayofweek()==4). You should see 32 records in total, 25 of it is not null.

(additional: you can read about S-1 IPO filing to understand the context)

In [ ]:
# Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = 'https://stockanalysis.com/ipos/filings/'

response = requests.get(url, headers=headers)

filings_table = pd.read_html(response.text)
df = filings_table[0]

In [ ]:
# Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs).

df['Filing Date'] = pd.to_datetime(df['Filing Date'], format="mixed")
df['Shares Offered'] = df['Shares Offered'].replace('-', np.NaN)
df['Shares Offered'] = pd.to_numeric(df['Shares Offered']).astype(float)


# Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified,
# to the price (if only one number is provided), or to the average of 2 prices (if a range is given).
# You may be inspired by the function extract_numbers() in [Code Snippet 4], or you can write your own function to "parse" a string.

def set_avg_price(input_string):
    temp = re.findall(r'\d+.\d+', input_string)
    if temp == []:
        return np.NaN
    else:
        r = 0
        for i in temp:
            r += float(i)
        return (r / len(temp))


# Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

def set_shares_offered_value(shares_offered, avg_price):
    series = pd.Series([shares_offered, avg_price])
    if series.isnull().any().any():
        return np.NaN
    else:
        return shares_offered * avg_price


df['Avg_price'] = df['Price Range'].apply(set_avg_price)
df["Shares_offered_value"] = df.apply(lambda x: set_shares_offered_value(x['Shares Offered'], x['Avg_price']), axis=1)

In [ ]:
# Find the total sum in $m (millions of USD, closest INTEGER number) for all filings during 2023, which happened on Fridays (Date.dt.dayofweek()==4).
# You should see 32 records in total, 25 of it is not null.

fridays = df[(df["Filing Date"].dt.year == 2023) & (df["Filing Date"].dt.dayofweek == 4)]

(fridays["Shares_offered_value"].sum() / 1000000).round(0)

# Question 2: IPOs "Fixed days hold" strategy
Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?

Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/). Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks. Also, you can see the ticker changes using this link. Some of the tickers (like 'DYCQ' and 'LEGT') were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day). Find X, when the 75% quantile growth (among 185 investments) is the highest.

HINTs:

You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
You can use the DataFrame.describe() function to get mean, min, max, 25-50-75% quantiles.
Additional:

You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
What's your recommendation: Do you suggest pursuing this strategy for an optimal X?

In [ ]:
# Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/).

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url_2023 = 'https://stockanalysis.com/ipos/2023/'
url_2024 = 'https://stockanalysis.com/ipos/2024/'


def get_data(headers, url):
    response = requests.get(url, headers=headers)
    filings_table = pd.read_html(response.text)
    return filings_table[0]


ipos_2023 = get_data(headers, url_2023)
ipos_2024 = get_data(headers, url_2024)
stacked_ipos_df = pd.concat([ipos_2023, ipos_2024], ignore_index=True)

In [ ]:
stacked_ipos_df[stacked_ipos_df.Symbol.isin(['IBAC', 'BKHA'])]

In [ ]:
# convert to datetime

stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'])

In [ ]:
TICKERS = [e for e in stacked_ipos_df[stacked_ipos_df['IPO Date'] < '2024-03-01'].Symbol.tolist()]
len(TICKERS)

In [ ]:
# Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks.
# Also, you can see the ticker changes using this link. Some of the tickers (like 'DYCQ' and 'LEGT') were on the market less than 30 days (11 and 21 days, respectively).
# Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

def correct_ticker(ticker: str):
    match ticker:
        case 'IBAC':
            return 'IBACU'
        case 'BKHA':
            return 'BKHAU'
        case 'PTHR':
            return 'PTHRU'
        case _:
            return ticker

In [ ]:
# Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB').
# Please remove 'RYZB', as it is no longer available on Yahoo Finance.

def get_stocks_df(TICKERS):
    stocks_df = pd.DataFrame({'A': []})
    print(stocks_df)

    for i, ticker in enumerate(TICKERS):
        if ticker == 'RYZB':
            continue
        ticker = correct_ticker(ticker)
        print(f'Getting {ticker}')
        print(i, ticker)

        # work with stock prices
        historyPrices = yf.download(tickers=ticker, period='max', interval='1d')

        # generate column for historical prices that we want to predict
        historyPrices['Ticker'] = ticker
        historyPrices['Year'] = historyPrices.index.year
        historyPrices['Month'] = historyPrices.index.month
        historyPrices['Weekday'] = historyPrices.index.weekday
        historyPrices['Date'] = historyPrices.index.date

        # historical returns
        for i in [1, 3, 7, 30, 90, 365]:
            historyPrices['growth_' + str(i) + 'd'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)

        historyPrices['growth_5d'] = historyPrices['Adj Close'].shift(-55) / historyPrices['Adj Close']

        # Technical indicators
        # SimpleMovinAverage 10 days and 20 days
        historyPrices['SMA10'] = historyPrices['Adj Close'].rolling(window=10).mean()
        historyPrices['SMA20'] = historyPrices['Adj Close'].rolling(window=20).mean()
        historyPrices['growing_movin_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
        historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Adj Close']

        # 30d rolling volatility
        historyPrices['volatility'] = historyPrices['Adj Close'].rolling(window=30).std() * np.sqrt(252)

        # what we wants to predict
        historyPrices['is_positive_growth_5d_in_future'] = np.where(historyPrices['growth_5d'] > 1, 1, 0)

        # sleep 1 sec between downloads - not to overload API server

        time.sleep(1)

        if stocks_df.empty:
            stocks_df = historyPrices
        else:
            stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

    return stocks_df

In [ ]:
stocks_df = get_stocks_df(TICKERS)
stocks_df.info()

In [ ]:
stocks_df.Ticker.nunique()

In [ ]:
stocks_df.head()

In [ ]:
df = stocks_df.copy(deep=True)

In [ ]:
for i in range(30):
    shift = i + 1
    df[f'future_growth_' + str(shift) + '_d'] = df['Adj Close'].shift(-shift) / df['Adj Close']

In [ ]:
# min_dates['Ticker'] - min_dates.index

min_dates = pd.DataFrame(df.groupby('Ticker').Date.min())
min_dates.head()

In [ ]:
min_dates.reset_index(inplace=True)
min_dates

In [ ]:
merged = pd.merge(df, min_dates, on=['Date', 'Ticker'], how='inner')
merged

In [ ]:
KEYS = [k for k in merged.keys() if k.find('future_') == 0]
KEYS

In [ ]:
merged[KEYS].describe().T

In [ ]:
merged[merged.future_growth_30_d > 9]

In [ ]:
merged[merged.future_growth_30_d > 9].Date

In [ ]:
# interested in 75 % only
dct = dict(merged[KEYS].describe().T['75%'])

In [ ]:
future_growth = dct

In [ ]:
sorted_dict = dict(sorted(future_growth.items(), key=lambda item: item[1]))
sorted_dict

In [ ]:
# Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest
list(sorted_dict.keys())[-1]


# Question 3: Is Growth Concentrated in the Largest Stocks?
Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?

Reuse [Code Snippet 7] to obtain OHLCV stats for 33 stocks for 10 full years of data (2014-01-01 to 2023-12-31). You'll need to download slightly more data (7 periods before 2014-01-01 to calculate the growth_7d for the first 6 days correctly):

US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

Now let's add the top 12-22 stocks (as of end-April 2024):

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS (from the actual stats on Yahoo Finance)

Calculate growth_7d for every stock and every day. Get the average daily growth_7d for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

For example, for the first of data you should have:

Date	ticker_category	growth_7d
2014-01-01	LARGE	1.011684
2014-01-01	LARGEST	1.011797
On that day, the LARGEST group was growing faster than LARGE one (new stocks).

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

HINT: you can use pandas.pivot_table() to "flatten" the table (LARGE and LARGEST growth_7d as columns)

In [ ]:
# Reuse [Code Snippet 7] to obtain OHLCV stats for 33 stocks for 10 full years of data (2014-01-01 to 2023-12-31).
# You'll need to download slightly more data (7 periods before 2014-01-01 to calculate the growth_7d for the first 6 days correctly):

US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO', 'V', 'JPM']

EU_STOCKS = ['NVO', 'MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE', 'IDEXY', 'CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS', 'TCS.NS', 'HDB', 'BHARTIARTL.NS', 'IBN', 'SBIN.NS', 'LICI.NS', 'INFY', 'ITC.NS',
                'HINDUNILVR.NS', 'LT.NS']

PREVIOUS_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

NEW_US = ['TSLA', 'WMT', 'XOM', 'UNH', 'MA', 'PG', 'JNJ', 'MRK', 'HD', 'COST', 'ORCL']

NEW_EU = ['PRX.AS', 'AIR.PA', 'SU.PA', 'ETN', 'SNY', 'BUD', 'DTE.DE', 'ALV.DE', 'MDT', 'AI.PA', 'EL.PA']

NEW_INDIA = ['BAJFINANCE.NS', 'MARUTI.NS', 'HCLTECH.NS', 'TATAMOTORS.NS', 'SUNPHARMA.NS', 'ONGC.NS', 'ADANIENT.NS',
             'NTPC.NS', 'KOTAKBANK.NS', 'TITAN.NS']

ADDITIONAL_STOCKS = NEW_US + NEW_EU + NEW_INDIA

ALL_STOCKS = PREVIOUS_STOCKS + ADDITIONAL_STOCKS

In [ ]:
# Reuse [Code Snippet 7] to obtain OHLCV stats for 33 stocks for 10 full years of data (2014-01-01 to 2023-12-31).

full_df = get_stocks_df(ALL_STOCKS)

In [ ]:
full_df.info()

In [ ]:
full_df.Date = pd.to_datetime(full_df.Date)

In [ ]:
full_df_short = full_df[(full_df.Date > '2014-01-01') & (full_df.Date <= '2023-12-31')]
full_df_short.info()

In [ ]:
full_df_short.Ticker.nunique()

In [ ]:
full_df_short.loc[:, 'ticker_category'] = full_df_short.Ticker.apply(lambda x: 'OLD' if x in PREVIOUS_STOCKS else 'NEW')

In [ ]:
# You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS (from the actual stats on Yahoo Finance)

full_df_short.groupby('ticker_category').Ticker.nunique()

In [ ]:
daily_avg_growth = full_df_short.groupby(['Date', 'ticker_category']).growth_7d.mean().reset_index()
daily_avg_growth

In [47]:
# Calculate growth_7d for every stock and every day. 

pivot_table = daily_avg_growth.pivot_table(index=['Date'], columns='ticker_category', values='growth_7d', aggfunc='mean').reset_index()
pivot_table

ticker_category,Date,NEW,OLD
0,2014-01-02,1.004176,1.006333
1,2014-01-03,1.002944,0.999152
2,2014-01-06,0.997761,0.994203
3,2014-01-07,0.994872,0.992509
4,2014-01-08,0.992185,0.987730
...,...,...,...
2589,2023-12-22,1.003994,1.013788
2590,2023-12-26,1.014441,1.014900
2591,2023-12-27,1.004676,1.014155
2592,2023-12-28,1.000828,1.008533


In [48]:
# Get the average daily growth_7d for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

pivot_table['more_new'] = np.where(pivot_table['NEW'] > pivot_table['OLD'], 1, 0)
pivot_table

ticker_category,Date,NEW,OLD,more_new
0,2014-01-02,1.004176,1.006333,0
1,2014-01-03,1.002944,0.999152,1
2,2014-01-06,0.997761,0.994203,1
3,2014-01-07,0.994872,0.992509,1
4,2014-01-08,0.992185,0.987730,1
...,...,...,...,...
2589,2023-12-22,1.003994,1.013788,0
2590,2023-12-26,1.014441,1.014900,0
2591,2023-12-27,1.004676,1.014155,0
2592,2023-12-28,1.000828,1.008533,0


In [49]:
# Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value).

pivot_table.more_new.value_counts()

more_new
0    1384
1    1210
Name: count, dtype: int64

In [50]:
pivot_table.more_new.value_counts() / len(pivot_table)

more_new
0    0.533539
1    0.466461
Name: count, dtype: float64

In [51]:
np.round(100*pivot_table.more_new.value_counts()/len(pivot_table))

more_new
0    53.0
1    47.0
Name: count, dtype: float64

# Question 4: Trying Another Technical Indicators strategy
What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?

First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31). If you encounter any difficulties running the Colab - you can download it using this link.

Let's assume you've learned about the awesome CCI indicator (Commodity Channel Index), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (Date.dt.dayofweek()==4).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

What's the expected gross profit (no fees) that you get in THOUSANDS $ (closest integer value) over many operations in 10 years? One operation calculations: if you invested $1000 and received $1010 in 5 days - you add $10 to gross profit, if you received $980 - add -$20 to gross profit. You need to sum these results over all trades (460 times in 10 years).